In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm

from matplotlib import rc
rc('font',family='AppleGothic')

import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
from sklearn import linear_model
from scipy import stats
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
import glob
import os

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import urllib.request
import ssl
import requests
from urllib.request import urlopen
from selenium import webdriver
import time
import re


In [2]:
### 각 연도별 데이터에서 필요 없는 행 열 제거 후 저장

def preprocessing(*yeardatas):
    for yeardata in yeardatas:
        data = pd.read_csv('../data/Kofic_movie_Data/Kofic_Original Data/KOBIS_연도별박스오피스_'+yeardata+'년도.csv')
        data = data.drop(index=[0, 1, 2], axis=0)
        data = data.rename(columns = data.iloc[0])
        data = data.drop(data.index[0])
        data = data.drop(columns = ['순위'], axis = 1)
        data = data.drop(columns = ['매출액 점유율'], axis = 1)
        data = data.drop(columns = ['국적'], axis = 1)
        data = data.reset_index(drop = True)
        data = data.drop(index = [len(data)-1], axis = 0)
        data = data.head(100)
        data.to_csv('../data/Kofic_movie_Data/Kofic_Merge Data/'+yeardata+'.csv',index=False)


In [3]:
preprocessing('2010', '2011', '2012', '2013','2014', '2015', '2016', '2017', '2018', '2019', '2020')

In [4]:
### 각 csv파일 합치기

input_data = '../data/Kofic_movie_Data/Kofic_Merge Data'
output_data = '../data/Kofic_movie_Data/Kofic_Merge Data/2010~2020.csv'

data_list = glob.glob(os.path.join(input_data, '20*'))
all_data=[]
for file in data_list:
    df = pd.read_csv(file) 
    all_data.append(df)

combinedata = pd.concat(all_data, axis=0, ignore_index=True)
combinedata = combinedata.drop_duplicates(['영화명'])
combinedata.reset_index(drop = True,inplace = True)

In [5]:
### 저장

combinedata.to_csv('../data/Kofic_movie_Data/Kofic_Merge Data/2010~2020.csv',index=False)
combinedata.to_excel('../data/Kofic_movie_Data/Kofic_Merge Data/2010~2020.xlsx',index=False)


In [7]:
### 불러올 때 매출액, 관객수, 스크린수, 상영횟수 int형 변환 
combinedata = pd.read_csv('../data/Kofic_movie_Data/Kofic_Merge Data/2010~2020.csv',thousands=',')
combinedata

,영화명,개봉일,매출액,관객수,스크린수,상영횟수,대표국적,배급사
0,아바타,2009.12.17,81455728000,8151952,714,111860,미국,"주식회사 해리슨앤컴퍼니,이십세기폭스코리아(주)"
1,아저씨,2010.8.4,47101332000,6178248,501,109853,한국,CJ ENM
2,인셉션,2010.7.21,43394990000,5827444,531,81891,미국,"(주)디스테이션,워너브러더스 코리아(주)"
3,의형제,2010.2.4,40153729000,5416812,569,118243,한국,(주)쇼박스
4,아이언맨 2,2010.4.29,32635195500,4425003,921,87665,미국,CJ ENM
...,...,...,...,...,...,...,...,...
1042,더 프린세스 : 도둑맞은 공주,2020.8.12,457449660,55828,423,4192,우크라이나,(주)팝엔터테인먼트
1043,극장판 바이올렛 에버가든,2020.11.12,467986490,55030,131,5063,일본,(주)라이크콘텐츠
1044,나의 첫 번째 슈퍼스타,2020.6.10,474367360,53748,462,8832,미국,유니버설픽쳐스인터내셔널 코리아(유)
1045,프리키 데스데이,2020.11.25,451225690,51812,631,12882,미국,유니버설픽쳐스인터내셔널 코리아(유)


In [8]:
combinedata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1047 entries, 0 to 1046
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   영화명     1047 non-null   object
 1   개봉일     1047 non-null   object
 2   매출액     1047 non-null   int64 
 3   관객수     1047 non-null   int64 
 4   스크린수    1047 non-null   int64 
 5   상영횟수    1047 non-null   int64 
 6   대표국적    1047 non-null   object
 7   배급사     1047 non-null   object
dtypes: int64(4), object(4)
memory usage: 65.6+ KB


In [11]:
combinedata['대표국적'].value_counts()

미국       478
한국       476
일본        34
영국        16
프랑스       11
중국         6
독일         4
벨기에        4
러시아        4
인도         3
스페인        3
대만         3
홍콩         2
아이슬란드      1
아일랜드       1
우크라이나      1
Name: 대표국적, dtype: int64

In [12]:
movie_list=combinedata['영화명']
movie_list

0                    아바타
1                    아저씨
2                    인셉션
3                    의형제
4                 아이언맨 2
              ...       
1042    더 프린세스 : 도둑맞은 공주
1043       극장판 바이올렛 에버가든
1044        나의 첫 번째 슈퍼스타
1045            프리키 데스데이
1046              다크 나이트
Name: 영화명, Length: 1047, dtype: object

In [13]:
### 영화 이름, 장르 , 감독, 주연배우, 영화등급 crawling 

name_list = []
genre_list = []
director_list = []
actor_list = []
film_rank_list = []


In [17]:
## 특정 url로 request하거나 urlopen시 아래와 같은 에러가 발생하는 경우해결
ssl._create_default_https_context = ssl._create_unverified_context

# driver = webdriver.Chrome('/Users/hanjaemin/PycharmProjects/acorn/chromedriver')
driver = webdriver.Chrome('../../chromedriver')
driver.get("https://movie.naver.com/")

C:\Users\PC\AppData\Local\Temp/ipykernel_12024/1485753734.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('../..//chromedriver')


In [18]:
for movie in movie_list:
    elem = driver.find_element_by_id("ipt_tx_srch")
    elem.send_keys(movie)
    time.sleep(1)
    try:
        path = driver.find_element_by_xpath('//*[@id="jAutoMV"]/ul/li[1]/a')
        path.click()
        
    except:
        elem.send_keys(Keys.RETURN)
        driver.find_element_by_xpath('//*[@id="old_content"]/ul[2]/li[1]/dl/dt/a').click()

    name = driver.find_elements_by_css_selector('#content > div.article > div.mv_info_area > div.mv_info > h3 > a')

    genre = driver.find_elements_by_css_selector('#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(2) > p > span:nth-child(1) > a:nth-child(1)')

    actor = driver.find_elements_by_css_selector('#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(6) > p')

    director = driver.find_elements_by_css_selector('#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(4) > p > a')

    film_rank = driver.find_elements_by_css_selector('#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-child(8) > p')


    if len(name) == 0 :
        name_list.append('')
    else:
        for i in range(len(name)):
            name_list.append(name[i].text)


    if len(genre) == 0 :
        genre_list.append('')
    else:
        for i in range(len(genre)):
            genre_list.append(genre[i].text)


    if len(actor) == 0 :
        actor_list.append('')
    else:
        for i in range(len(actor)):
            actor_list.append(actor[i].text)


    if len(director) == 0 :
        director_list.append('')
    else:
        for i in range(len(director)):
            if i == 1:
                break
            director_list.append(director[i].text)


    if len(film_rank) == 0 :
        film_rank_list.append('')
    else:
        for i in range(len(film_rank)):
            film_rank_list.append(film_rank[i].text)




C:\Users\PC\AppData\Local\Temp/ipykernel_12024/3434358419.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  elem = driver.find_element_by_id("ipt_tx_srch")
C:\Users\PC\AppData\Local\Temp/ipykernel_12024/3434358419.py:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  path = driver.find_element_by_xpath('//*[@id="jAutoMV"]/ul/li[1]/a')
C:\Users\PC\AppData\Local\Temp/ipykernel_12024/3434358419.py:13: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  name = driver.find_elements_by_css_selector('#content > div.article > div.mv_info_area > div.mv_info > h3 > a')
C:\Users\PC\AppData\Local\Temp/ipykernel_12024/3434358419.py:15: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  genre = driver.find_elements_by_css_selector('#content > div.article > div.mv_info_area > div.mv_info > dl > dd:n

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=97.0.4692.71)
Stacktrace:
Backtrace:
	Ordinal0 [0x011EFDC3+2555331]
	Ordinal0 [0x011877F1+2127857]
	Ordinal0 [0x01082E08+1060360]
	Ordinal0 [0x01085524+1070372]
	Ordinal0 [0x010853EE+1070062]
	Ordinal0 [0x01085650+1070672]
	Ordinal0 [0x010AEEB9+1240761]
	Ordinal0 [0x010AEF94+1240980]
	Ordinal0 [0x010A76AD+1210029]
	Ordinal0 [0x010C7B0C+1342220]
	Ordinal0 [0x010A4984+1198468]
	Ordinal0 [0x010C7C14+1342484]
	Ordinal0 [0x010D75FA+1406458]
	Ordinal0 [0x010C7976+1341814]
	Ordinal0 [0x010A36B6+1193654]
	Ordinal0 [0x010A4546+1197382]
	GetHandleVerifier [0x01389622+1619522]
	GetHandleVerifier [0x0143882C+2336844]
	GetHandleVerifier [0x012823E1+541697]
	GetHandleVerifier [0x01281443+537699]
	Ordinal0 [0x0118D18E+2150798]
	Ordinal0 [0x01191518+2168088]
	Ordinal0 [0x01191660+2168416]
	Ordinal0 [0x0119B330+2208560]
	BaseThreadInitThunk [0x760CFA29+25]
	RtlGetAppCon

In [24]:
len(film_rank_list)

1047

In [41]:
len(name_list)

1047

In [26]:
len(genre_list)

1047

In [27]:
len(director_list)

1047

In [28]:
len(genre_list)

1047

In [29]:
len(actor_list)

1047

In [49]:
data={
    'director': director_list,
    'genre': genre_list,
    'actor': actor_list,
    'filmrank': film_rank_list
}
crawling_data=pd.DataFrame(data)

In [50]:
crawling_data

,director,genre,actor,filmrank
0,이병헌,코미디,"류승룡(고반장), 이하늬(장형사), 진선규(마형사)",[국내] 15세 관람가
1,정세교,다큐멘터리,"김한민, 오타니 료헤이, 이해영",[국내] 12세 관람가
2,안소니 루소,액션,"로버트 다우니 주니어(토니 스타크 / 아이언맨), 크리스 에반스(스티브 로저스 / ...",[국내] 12세 관람가
3,류승완,액션,"황정민(서도철), 유아인(조태오), 유해진(최상무)",[국내] 15세 관람가
4,김용화,판타지,"하정우(강림), 주지훈(해원맥), 김향기(덕춘)",[국내] 12세 관람가
...,...,...,...,...
1042,올레 말라므,애니메이션,"사문영(밀라 목소리), 남도형(루슬란 목소리), 서반석(레스터 목소리)",[국내] 전체 관람가
1043,크리스토퍼 랜던,코미디,"빈스 본, 캐서린 뉴튼",[국내] 15세 관람가
1044,신카이 마코토,애니메이션,"심규혁(호다카), 김유림(히나), 최한(스가)",[국내] 15세 관람가
1045,로버트 렌스,애니메이션,"양정화(눈의 여왕 한국어 목소리), 박지윤(겔다 한국어 목소리), 임채헌(헤럴드 한...",[국내] 전체 관람가


In [51]:
crawling_data.to_csv('/Users/hanjaemin/Desktop/final_project/BigData_Final_Project/data/Naver_Crawling_Data/crawling_data.csv',index=False)
crawling_data.to_excel('/Users/hanjaemin/Desktop/final_project/BigData_Final_Project/data/Naver_Crawling_Data/crawling_data.xlsx',index=False)

In [138]:
crawling_data = pd.read_csv('/Users/hanjaemin/Desktop/final_project/BigData_Final_Project/data/Naver_Crawling_Data/crawling_data.csv')
crawling_data

,director,genre,actor,filmrank
0,이병헌,코미디,"류승룡(고반장), 이하늬(장형사), 진선규(마형사)",[국내] 15세 관람가
1,정세교,다큐멘터리,"김한민, 오타니 료헤이, 이해영",[국내] 12세 관람가
2,안소니 루소,액션,"로버트 다우니 주니어(토니 스타크 / 아이언맨), 크리스 에반스(스티브 로저스 / ...",[국내] 12세 관람가
3,류승완,액션,"황정민(서도철), 유아인(조태오), 유해진(최상무)",[국내] 15세 관람가
4,김용화,판타지,"하정우(강림), 주지훈(해원맥), 김향기(덕춘)",[국내] 12세 관람가
...,...,...,...,...
1042,올레 말라므,애니메이션,"사문영(밀라 목소리), 남도형(루슬란 목소리), 서반석(레스터 목소리)",[국내] 전체 관람가
1043,크리스토퍼 랜던,코미디,"빈스 본, 캐서린 뉴튼",[국내] 15세 관람가
1044,신카이 마코토,애니메이션,"심규혁(호다카), 김유림(히나), 최한(스가)",[국내] 15세 관람가
1045,로버트 렌스,애니메이션,"양정화(눈의 여왕 한국어 목소리), 박지윤(겔다 한국어 목소리), 임채헌(헤럴드 한...",[국내] 전체 관람가


In [139]:
final_data = pd.concat([combinedata, crawling_data], axis=1)

In [19]:
final_data = pd.read_csv('../../d.csv')

In [20]:
final_data.head()

,영화명,개봉일,매출액,관객수,스크린수,상영횟수,대표국적,배급사,director,genre,actor,filmrank
0,극한직업,2019.1.23,139651845516,16265618,2003,292584,한국,(주)씨제이이엔엠,이병헌,코미디,"류승룡(고반장), 이하늬(장형사), 진선규(마형사)",[국내] 15세 관람가
1,명량,2014.7.30,135748398910,17613682,1587,188611,한국,(주)씨제이이엔엠,정세교,다큐멘터리,"김한민, 오타니 료헤이, 이해영",[국내] 12세 관람가
2,어벤져스: 엔드게임,2019.4.24,122182694160,13934592,2835,242001,미국,월트디즈니컴퍼니코리아 유한책임회사,안소니 루소,액션,"로버트 다우니 주니어(토니 스타크 / 아이언맨), 크리스 에반스(스티브 로저스 / ...",[국내] 12세 관람가
3,베테랑,2015.8.5,105168155250,13414009,1115,199231,한국,(주)씨제이이엔엠,류승완,액션,"황정민(서도철), 유아인(조태오), 유해진(최상무)",[국내] 15세 관람가
4,신과함께-인과 연,2018.8.1,102666146909,12274996,2235,179981,한국,롯데컬처웍스(주)롯데엔터테인먼트,김용화,판타지,"하정우(강림), 주지훈(해원맥), 김향기(덕춘)",[국내] 12세 관람가


In [21]:
### 열이름 변경
final_data.rename(columns={'영화명': 'moviename',
                          '개봉일' : 'openingdate',
                          '매출액' : 'sales',
                          '관객수' : 'attendance',
                          '스크린수' : 'screen',
                          '상영횟수' : 'playcount',
                          '대표국적' : 'country',
                          '배급사' : 'distributor'},inplace = True)

In [22]:
### 누적매출액 순으로 정렬

final_data.sort_values(['sales'], ascending = False, inplace = True)
final_data.reset_index(drop = True, inplace=True)
final_data

,moviename,openingdate,sales,attendance,screen,playcount,country,distributor,director,genre,actor,filmrank
0,극한직업,2019.1.23,139651845516,16265618,2003,292584,한국,(주)씨제이이엔엠,이병헌,코미디,"류승룡(고반장), 이하늬(장형사), 진선규(마형사)",[국내] 15세 관람가
1,명량,2014.7.30,135748398910,17613682,1587,188611,한국,(주)씨제이이엔엠,정세교,다큐멘터리,"김한민, 오타니 료헤이, 이해영",[국내] 12세 관람가
2,어벤져스: 엔드게임,2019.4.24,122182694160,13934592,2835,242001,미국,월트디즈니컴퍼니코리아 유한책임회사,안소니 루소,액션,"로버트 다우니 주니어(토니 스타크 / 아이언맨), 크리스 에반스(스티브 로저스 / ...",[국내] 12세 관람가
3,베테랑,2015.8.5,105168155250,13414009,1115,199231,한국,(주)씨제이이엔엠,류승완,액션,"황정민(서도철), 유아인(조태오), 유해진(최상무)",[국내] 15세 관람가
4,신과함께-인과 연,2018.8.1,102666146909,12274996,2235,179981,한국,롯데컬처웍스(주)롯데엔터테인먼트,김용화,판타지,"하정우(강림), 주지훈(해원맥), 김향기(덕춘)",[국내] 12세 관람가
...,...,...,...,...,...,...,...,...,...,...,...,...
1042,더 프린세스 : 도둑맞은 공주,2020.8.12,457449660,55828,423,4192,우크라이나,(주)팝엔터테인먼트,올레 말라므,애니메이션,"사문영(밀라 목소리), 남도형(루슬란 목소리), 서반석(레스터 목소리)",[국내] 전체 관람가
1043,프리키 데스데이,2020.11.25,451225690,51812,631,12882,미국,유니버설픽쳐스인터내셔널 코리아(유),크리스토퍼 랜던,코미디,"빈스 본, 캐서린 뉴튼",[국내] 15세 관람가
1044,날씨의 아이,2019.10.30,444956180,58968,194,5921,일본,"메가박스중앙(주)플러스엠,(주)미디어캐슬,워터홀컴퍼니(주)",신카이 마코토,애니메이션,"심규혁(호다카), 김유림(히나), 최한(스가)",[국내] 15세 관람가
1045,눈의 여왕4,2019.12.24,444649620,67159,271,2239,러시아,(주)넥스트엔터테인먼트월드(NEW),로버트 렌스,애니메이션,"양정화(눈의 여왕 한국어 목소리), 박지윤(겔다 한국어 목소리), 임채헌(헤럴드 한...",[국내] 전체 관람가


In [23]:
final_data.to_csv('../data/Merge_Data/merge_data.csv',index=False)
final_data.to_excel('../data/Merge_Data/merge_data.xlsx',index=False)